In [ ]:
import os
import pandas as pd
cities = pd.read_csv('cities.csv', index_col=['CityId'], nrows=None)
cities = cities * 1000 
cities.head()

In [ ]:
%%bash -e
wget http://akira.ruc.dk/~keld/research/LKH/LKH-2.0.9.tgz
tar xvfz LKH-2.0.9.tgz
cd LKH-2.0.9
make

In [7]:
def write_tsp(nodes, filename, name='traveling-santa-2018-prime-paths'):
    # From https://www.kaggle.com/blacksix/concorde-for-5-hours.
    with open(filename, 'w') as f:
        f.write('NAME : %s\n' % name)
        f.write('COMMENT : %s\n' % name)
        f.write('TYPE : TSP\n')
        f.write('DIMENSION : %d\n' % len(cities))
        f.write('EDGE_WEIGHT_TYPE : EUC_2D\n')
        f.write('NODE_COORD_SECTION\n')
        for row in cities.itertuples():
            f.write('%d %.11f %.11f\n' % (row.Index + 1, row.X, row.Y))
        f.write('EOF\n')

write_tsp(cities, 'cities.tsp')

In [20]:
def write_parameters(parameters, filename='LKH-2.0.9/params.par'):
    with open(filename, 'w') as f:
        for param, value in parameters:
            f.write("{} = {}\n".format(param, value))
    print("Parameters saved as", filename)

parameters = [
    ("PROBLEM_FILE", "cities.tsp"),
    ("OUTPUT_TOUR_FILE", "tsp_solution.csv"),
    ("SEED", 2018),
    ('CANDIDATE_SET_TYPE', 'POPMUSIC'), #'NEAREST-NEIGHBOR', 'ALPHA'),
    ('INITIAL_PERIOD', 10000),
    ('MAX_TRIALS', 1000),
]
write_parameters(parameters)

('Parameters saved as', 'LKH-2.0.9/params.par')


In [21]:
%%bash -e
timeout 100s ./LKH-2.0.9/LKH ./LKH-2.0.9/params.par

PARAMETER_FILE = ./LKH-2.0.9/params.par
Reading PROBLEM_FILE: "cities.tsp" ... done
ASCENT_CANDIDATES = 50
BACKBONE_TRIALS = 0
BACKTRACKING = NO
# CANDIDATE_FILE =
CANDIDATE_SET_TYPE = POPMUSIC
# EDGE_FILE =
EXCESS = 5.0564e-06
EXTRA_CANDIDATES = 0 
EXTRA_CANDIDATE_SET_TYPE = QUADRANT
GAIN23 = YES
GAIN_CRITERION = YES
INITIAL_PERIOD = 10000
INITIAL_STEP_SIZE = 1
INITIAL_TOUR_ALGORITHM = WALK
# INITIAL_TOUR_FILE = 
INITIAL_TOUR_FRACTION = 1.000
# INPUT_TOUR_FILE = 
KICK_TYPE = 0
KICKS = 1
# MAX_BREADTH =
MAX_CANDIDATES = 5 
MAX_SWAPS = 197769
MAX_TRIALS = 1000
# MERGE_TOUR_FILE =
MOVE_TYPE = 5
# NONSEQUENTIAL_MOVE_TYPE = 5
# OPTIMUM =
OUTPUT_TOUR_FILE = tsp_solution.csv
PATCHING_A = 1 
PATCHING_C = 0 
# PI_FILE = 
POPMUSIC_INITIAL_TOUR = NO
POPMUSIC_MAX_NEIGHBORS = 5
POPMUSIC_SAMPLE_SIZE = 10
POPMUSIC_SOLUTIONS = 50
POPMUSIC_TRIALS = 1
# POPULATION_SIZE = 0
PRECISION = 100
PROBLEM_FILE = cities.tsp
RECOMBINATION = IPT
RESTRICTED_SEARCH = YES
RUNS = 10
SEED = 2018
STOP_AT_OPTIMUM = YES
S

In [17]:
def read_tour(filename):
    tour = []
    for line in open(filename).readlines():
        line = line.replace('\n', '')
        try:
            tour.append(int(line) - 1)
        except ValueError as e:
            pass  # skip if not a city id (int)
    return tour[:-1]

tour = read_tour('tsp_solution.csv')
print("Tour length", len(tour))

IOError: [Errno 2] No such file or directory: 'tsp_solution.csv'